In [ ]:
import pandas as pd
from flask import Flask, request

app = Flask(__name__)

@app.route('/upload', methods=['POST'])
def upload():
    # Check if a file was uploaded
    if 'file' not in request.files:
        return 'No file was uploaded.'

    file = request.files['file']
    # Check if the file has a CSV extension
    if file.filename.rsplit('.', 1)[1].lower() != 'csv':
        return 'Please upload a CSV file.'

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file)

    # Drop the columns
    df = df.drop(columns=['response', 'organizer', 'start', 'end'])

    # Remove duplicates
    df = df.drop_duplicates()

    # Create boolean to delete non recurring meetings + ones with attendee is 1
    mask = (df['attendees'] == 1) | (df['recurring'] == 'no')

    # drop rows captured by boolean
    df.drop(df[mask].index, inplace=True)

    # Drop the recurring column
    df = df.drop(columns=['recurring'])

    # Create a new  1:1's DataFrame with rows where attendees is equal to 2
    new_df = df[df['attendees'] == 2]

    # Move the new 1:1's dataframe to a new file
    new_df.to_csv('test2.csv', index=False, index_label='One on Ones')

    # Drop 1:1's from original file
    mask2 = (df['attendees'] == 2)

    df.drop(df[mask2].index, inplace=True)

    # Sort recurring meetings sheet by cadence
    df = df.sort_values(by='recurrence')

    # Sort 1:1's by cadence
    new_df = new_df.sort_values(by='recurrence')

    # Save 1:1's sheet
    new_df.to_csv('test2.csv', index=False)

    # Save recurring meetings sheet
    df.to_csv('test.csv', index=False)
    
    return 'File processed successfully.'

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Dec/2022 15:06:00] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [22/Dec/2022 15:06:00] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Dec/2022 15:06:13] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [22/Dec/2022 15:06:59] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [22/Dec/2022 15:07:05] "GET /upload HTTP/1.1" 405 -
127.0.0.1 - - [22/Dec/2022 15:07:05] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Dec/2022 15:07:50] "GET / HTTP/1.1" 404 -
